In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
#import seaborn as sns
from scipy import signal

import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
 
from mlprecip_utils import *
from mlprecip_models import *
from mlprecip_datasets import *
from mlprecip_xai import *
from mlprecip_plot import *

import mlprecip_datasets

from keras.models import load_model

import warnings

Using TensorFlow backend.
/homes/kpegion/.conda/envs/h5py-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/homes/kpegion/.conda/envs/h5py-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/homes/kpegion/.conda/envs/h5py-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/homes/kpegion

In [2]:
summer=[6,7,8]
sdate='1979-01-01'
edate='2018-12-31'
lat_reg=[24.0,36.0]
lon_reg=[269.0,283.0]
varname='precip'

In [3]:
fnames='/shared/ccsm4/kpegion/obs2/CPC-UNI/precip.V1.0.*.nc'
ds=xr.open_mfdataset(fnames,combine='by_coords').sel(time=slice(sdate,edate))

# Subset to SEUS
ds_seus=ds.sel(lat=slice(lat_reg[0],lat_reg[1]),lon=slice(lon_reg[0],lon_reg[1]))

# Calculate Anomalies
ds_anoms=ds_seus.groupby('time.dayofyear')-ds_seus.groupby('time.dayofyear').mean()

ds_anoms

<xarray.Dataset>
Dimensions:    (lat: 48, lon: 56, time: 14610)
Coordinates:
  * lat        (lat) float32 24.12 24.38 24.62 24.88 ... 35.12 35.38 35.62 35.88
  * lon        (lon) float32 269.1 269.4 269.6 269.9 ... 282.1 282.4 282.6 282.9
  * time       (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2018-12-31
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365
Data variables:
    precip     (time, lat, lon) float32 dask.array<chunksize=(1, 48, 56), meta=np.ndarray>

In [4]:
indices_dict=init_predictors_dict()

# Loop over seasons
for seas,slabel,seas_abbrv in zip([summer],['Summer'],['JJA']):

    print(slabel)
        
    features_list=[]
    
    # Loop over all indices
    for i,f in enumerate(indices_dict):

        # Read the Indices
        if (f['ptype']=='index' and f['freq']=='mon'):
            ds=getattr(mlprecip_datasets,f['readfunc'])(f['file'],f['name'],sdate,edate)
            # Linearly interpolate monthly indices to daily
            ds=ds.resample(time='1D').interpolate("linear").sel(time=slice(sdate,edate))
        elif (f['name']=='RMM_amp'):
            _,ds=getattr(mlprecip_datasets,f['readfunc'])(f['file'],sdate,edate)
        elif (f['name']=='RMM_phase'):
            ds,_=getattr(mlprecip_datasets,f['readfunc'])(f['file'],sdate,edate)
        elif (f['name']=='pnaregimes'): 
            ds=getattr(mlprecip_datasets,f['readfunc'])(f['file'],seas_abbrv,sdate,edate)
        elif (f['name']=='nash_amp'):
            _,ds=getattr(mlprecip_datasets,f['readfunc'])(f['file'],sdate,edate)
        elif (f['name']=='nash_phase'):
            ds,_=getattr(mlprecip_datasets,f['readfunc'])(f['file'],sdate,edate)
        elif (f['name']=='z500_local'):
            ds=getattr(mlprecip_datasets,f['readfunc'])(f['file'],lat_reg,lon_reg,sdate,edate)
        else:
            ds=getattr(mlprecip_datasets,f['readfunc'])(f['file'],sdate,edate)
        
        # Standardize indices that are not categorical
        if (f['ptype']=='index'):
            v=f['name']
            ds[v]=(ds[v]-ds[v].mean(dim='time'))/ds[v].std(dim='time')
        
        # Select season from predictor index and target index
        ds_i=ds.sel(time=ds['time.month'].isin(seas)) #.dropna(dim='time')        
        ds_p=ds_anoms.sel(time=ds_anoms['time.month'].isin(seas)).sel(time=slice(sdate,edate)) #.dropna(dim='time')
        
        # Append this predictor index to list of all predictors
        features_list.append(ds_i[f['name']])

#----- FIT MODELS AND TEST  -----------#
    
    # Define Features
    ds_features=xr.merge(features_list).dropna(dim='time')   
    
    # Define Index Target as Mean precip in region
    # Note this is currently designed as a diagnostic not a real prediction; add target lead for prediction
    ds_target=ds_p.mean(dim=['lat','lon'])
    
    # Make sure we have the same times for target and features
    ds_target,ds_features=xr.align(ds_target,ds_features,join='inner')
    
    # Center data
    median=np.percentile(ds_target[varname],50)
    ds_target[varname]=ds_target[varname]-median
        


Summer


In [5]:
# Setup Features (X) and Target (Y)
    
X=ds_features.to_stacked_array('features',sample_dims=['time'])
Y=make_ohe_thresh_med(ds_target[varname])

print('Check Features and Target Dimensions')
print('Features (X): ',X.shape)
print('Target (Y): ',Y.shape)

nsamples=X.shape[0]
nfeatures=X.shape[1]

print("Samples: ",nsamples)
print("Features: ", nfeatures)
    
# Create Train and Test Sets
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,train_size=0.8,shuffle=False)

ntrain=X_train.shape[0]
ntest=X_test.shape[0]

print('Training Size: ',ntrain)
print('Testing Size: ',ntest)    

Upper Cat:  1502
Lower Cat:  1502
Check Features and Target Dimensions
Features (X):  (3004, 10)
Target (Y):  (3004, 2)
Samples:  3004
Features:  10
Training Size:  2403
Testing Size:  601


In [6]:
mymodel = KerasClassifier(build_fn=logmodel_med(X_train.shape[1]),epochs=250,batch_size=25, verbose=0)
history=mymodel.fit(X_train,Y_train,validation_data=(X_test,Y_test))

2021-10-29 17:36:37.560133: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX
2021-10-29 17:36:37.565723: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2399995000 Hz
2021-10-29 17:36:37.566684: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x562b8e7121a0 executing computations on platform Host. Devices:
2021-10-29 17:36:37.566728: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>


In [11]:
for layer in mymodel.model.layers: print(layer.get_config(), layer.get_weights())

{'name': 'dense_1', 'trainable': True, 'batch_input_shape': (None, 10), 'dtype': 'float32', 'units': 2, 'activation': 'softmax', 'use_bias': True, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1.0, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None} [array([[-0.14008771, -0.17931725],
       [-0.1153006 ,  0.00707352],
       [-0.3455337 , -0.19139165],
       [ 0.19118786,  0.1153224 ],
       [-0.1721504 , -0.21906339],
       [ 0.20657791,  0.22046718],
       [-0.5836612 , -0.6664344 ],
       [ 0.38813832,  0.4131134 ],
       [ 0.22359106,  0.07897352],
       [ 0.14674751,  0.10426403]], dtype=float32), array([-0.07948778,  0.07948775], dtype=float32)]


In [7]:
mymodel.model.save('mymodel')

In [8]:
loaded_model=load_model('mymodel')

In [9]:
loaded_model

In [12]:
for layer in loaded_model.model.layers: print(layer.get_config(), layer.get_weights())

{'name': 'dense_1', 'trainable': True, 'batch_input_shape': (None, 10), 'dtype': 'float32', 'units': 2, 'activation': 'softmax', 'use_bias': True, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1.0, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None} [array([[-0.14008771, -0.17931725],
       [-0.1153006 ,  0.00707352],
       [-0.3455337 , -0.19139165],
       [ 0.19118786,  0.1153224 ],
       [-0.1721504 , -0.21906339],
       [ 0.20657791,  0.22046718],
       [-0.5836612 , -0.6664344 ],
       [ 0.38813832,  0.4131134 ],
       [ 0.22359106,  0.07897352],
       [ 0.14674751,  0.10426403]], dtype=float32), array([-0.07948778,  0.07948775], dtype=float32)]


/homes/kpegion/.conda/envs/h5py-test/lib/python3.7/site-packages/keras/engine/sequential.py:110: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '
